In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
# First Dataset [Use Zip Code]
# No geographic values missing, no shapefiles missing, post 2000

In [3]:
df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

In [4]:
# Defining conditions
condition1 = (df_geographic_unique['latitude'].notna() & 
              df_geographic_unique['censusBlockGroupFips'].notna() & 
              df_geographic_unique['countyCode'].isna() & 
              df_geographic_unique['reportedZipCode'].isna())

condition2 = (df_geographic_unique['latitude'].notna() & 
              df_geographic_unique['censusBlockGroupFips'].notna() & 
              df_geographic_unique['countyCode'].isna() & 
              df_geographic_unique['reportedZipCode'].notna() &
              (df_geographic_unique['yearOfLoss'] < 2000))

# Filtering the dataframe based on the combined conditions
df_geographic_unique = df_geographic_unique[condition1 | condition2]

In [5]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss
33198,FL,NaN,NaN,3.402974e+10,3.402974e+11,39.6,-74.2,1992
43591,FL,NaN,NaN,3.714192e+10,3.714192e+11,34.4,-77.7,1996
43805,FL,NaN,NaN,4.820145e+10,4.820145e+11,29.8,-95.6,1992
45239,FL,NaN,NaN,3.611901e+10,3.611901e+11,41.2,-73.8,1996
46081,FL,NaN,NaN,5.118702e+10,5.118702e+11,38.9,-78.3,1996


In [6]:
print(sum(df_geographic_unique['state'].isna()))
print(sum(df_geographic_unique['reportedZipCode'].isna()))
print(sum(df_geographic_unique['countyCode'].isna()))
print(sum(df_geographic_unique['censusTract'].isna()))
print(sum(df_geographic_unique['censusBlockGroupFips'].isna()))
print(sum(df_geographic_unique['latitude'].isna()))
print(sum(df_geographic_unique['yearOfLoss'].isna()))

0
3515
6677
0
0
0
0


In [7]:
#df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].dropna().astype(int).astype(str)
#df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

df_geographic_unique['censusTract'] = [str(int(float(i))) for i in df_geographic_unique['censusTract']]
df_geographic_unique['censusTract'] = [censusBG.zfill(11) for censusBG in df_geographic_unique['censusTract']]

In [8]:
# Define bins and labels for yearOfLoss_2000_2021
bins_2000_2021 = [0, 2000, 2010, 2020, 2030]
labels_2000_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_2000_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_2000_2021, labels=labels_2000_2021, right=False).astype(int)

# Define bins and labels for yearOfLoss_1990_2021
bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, 2030]
labels_1990_2021 = [1990, 2000, 2010, 2020]


#df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

#df_geographic_unique = df_geographic_unique.drop(columns='yearOfLoss')

In [9]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_2000_2021
33198,FL,NaN,NaN,34029738001,340297380013,39.6,-74.2,1992,1990
43591,FL,NaN,NaN,37141920202,371419202023,34.4,-77.7,1996,1990
43805,FL,NaN,NaN,48201450700,482014507002,29.8,-95.6,1992,1990
45239,FL,NaN,NaN,36119013103,361190131031,41.2,-73.8,1996,1990
46081,FL,NaN,NaN,51187020300,511870203004,38.9,-78.3,1996,1990


## Read shapefiles

In [10]:
states = pygris.states()

state_df = states[['STUSPS', 'NAME', 'geometry']]

Using the default year of 2021


In [12]:
#Checking if all states found in our dataset are in the US Census Bureau TIGER/Line and cartographic boundary shapefiles

unique_states = df_geographic_unique['state'].unique()
state_STUSPS_unique = state_df['STUSPS'].unique()

np.all(np.isin(unique_states, state_STUSPS_unique))

False

In [13]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [14]:
chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 320000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df = pd.concat(gdf_list, ignore_index=True)

In [15]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 100000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

zipcode_df['year'] = zipcode_df['year'].replace({2012: 2010, 2021: 2020})
zipcode_df = zipcode_df.drop_duplicates(subset=['ZIPcode', 'year'])

In [16]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

County_df['year'] = County_df['year'].replace({2012: 2010, 2021: 2020})
County_df = County_df.drop_duplicates(subset=['CountyID', 'year'])

In [17]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/Tract_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
Tract_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))
Tract_df['year'] = Tract_df['year'].replace({2012: 2010, 2021: 2020})
Tract_df = Tract_df.drop_duplicates(subset=['censusTractID', 'year'])

## Geometry Intersection creation

In [18]:
state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)
County_df.rename(columns={'geometry': 'geometry_county'}, inplace=True)
Tract_df.rename(columns={'geometry': 'geometry_tract'}, inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_21680\2417984689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)


In [19]:
df_geographic_unique

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_2000_2021
33198,FL,NaN,NaN,34029738001,340297380013,39.6,-74.2,1992,1990
43591,FL,NaN,NaN,37141920202,371419202023,34.4,-77.7,1996,1990
43805,FL,NaN,NaN,48201450700,482014507002,29.8,-95.6,1992,1990
45239,FL,NaN,NaN,36119013103,361190131031,41.2,-73.8,1996,1990
46081,FL,NaN,NaN,51187020300,511870203004,38.9,-78.3,1996,1990
...,...,...,...,...,...,...,...,...,...
2286203,SD,58201.0,NaN,38035010300,380350103001,47.9,-97.1,1997,1990
2287929,MN,58225.0,NaN,38067950500,380679505001,48.7,-97.3,1997,1990
2290452,PR,682.0,NaN,72097080400,720970804001,18.2,-67.2,1998,1990
2341976,PR,NaN,NaN,72111730400,721117304001,18.1,-66.7,1987,1990


In [20]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

In [21]:
# Initial mapping with multi-index
BG_df.set_index(['GEOID', 'year'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips', 'yearOfLoss_2000_2021']).index.map(BG_df['geometry_BG'])

# Find rows with NaN values 
na_rows = df_geographic_unique['geometry_BG'].isna()

# Reset index of BG_df
#df_single_index = BG_df.reset_index()

# Sort by 'year' in descending order
#df_single_index = df_single_index.sort_values(by='year', ascending=False)

# Drop duplicates based on the 'GEOID' column. Keep the first occurrence.
#df_single_index = df_single_index.drop_duplicates(subset='GEOID', keep='first')

# Map these rows using only the 'GEOID'
#df_geographic_unique.loc[na_rows, 'geometry_BG'] = df_geographic_unique.loc[na_rows, 'censusBlockGroupFips'].map(df_single_index.set_index('GEOID')['geometry_BG'])

# Resetting the index of BG_df (return to multi-index)
#BG_df.reset_index(inplace=True)

In [22]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'yearOfLoss_2000_2021']).index.map(zipcode_df['geometry_zipcode'])

# Find rows with NaN values 
na_rows = df_geographic_unique['geometry_zipcode'].isna()

# # Reset index of BG_df
# df_single_index = zipcode_df.reset_index()

# # Sort by 'year' in descending order
# df_single_index = df_single_index.sort_values(by='year', ascending=False)

# # Drop duplicates based on the 'GEOID' column. Keep the first occurrence.
# df_single_index = df_single_index.drop_duplicates(subset='ZIPcode', keep='first')

# # Map these rows using only the 'GEOID'
# df_geographic_unique.loc[na_rows, 'geometry_zipcode'] = df_geographic_unique.loc[na_rows, 'reportedZipCode'].map(df_single_index.set_index('ZIPcode')['geometry_zipcode'])

# # Resetting the index of BG_df (return to multi-index)
# zipcode_df.reset_index(inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_21680\252387345.py:18: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_geographic_unique.loc[na_rows, 'geometry_zipcode'] = df_geographic_unique.loc[na_rows, 'reportedZipCode'].map(df_single_index.set_index('ZIPcode')['geometry_zipcode'])


In [23]:
# Setting the multi-index on lat_long_df
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

# Resetting the index of lat_long_df (optional, but good practice)
state_df.reset_index(inplace=True)

In [24]:
# Initial mapping with multi-index
County_df.set_index(['CountyID', 'year'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode', 'yearOfLoss_2000_2021']).index.map(County_df['geometry_county'])

# Find rows with NaN values
na_rows = df_geographic_unique['geometry_county'].isna()

# Reset index of County_df
# df_single_index = County_df.reset_index()

# # Sort by 'year' in descending order
# df_single_index = df_single_index.sort_values(by='year', ascending=False)

# # Drop duplicates based on the 'countyCode' column. Keep the first occurrence.
# df_single_index = df_single_index.drop_duplicates(subset='CountyID', keep='first')

# # Map these rows using only the 'countyCode'
# df_geographic_unique.loc[na_rows, 'geometry_county'] = df_geographic_unique.loc[na_rows, 'countyCode'].map(df_single_index.set_index('CountyID')['geometry_county'])

# # Resetting the index of County_df (return to multi-index)
# County_df.reset_index(inplace=True)

In [25]:
# Initial mapping with multi-index
Tract_df.set_index(['censusTractID', 'year'], inplace=True)
df_geographic_unique['geometry_tract'] = df_geographic_unique.set_index(['censusTract', 'yearOfLoss_2000_2021']).index.map(Tract_df['geometry_tract'])

# Find rows with NaN values
na_rows = df_geographic_unique['geometry_tract'].isna()

# # Reset index of County_df
# df_single_index = Tract_df.reset_index()

# # Sort by 'year' in descending order
# df_single_index = df_single_index.sort_values(by='year', ascending=False)

# # Drop duplicates based on the 'countyCode' column. Keep the first occurrence.
# df_single_index = df_single_index.drop_duplicates(subset='censusTractID', keep='first')

# # Map these rows using only the 'countyCode'
# df_geographic_unique.loc[na_rows, 'geometry_tract'] = df_geographic_unique.loc[na_rows, 'censusTract'].map(df_single_index.set_index('censusTractID')['geometry_tract'])

# # Resetting the index of County_df (return to multi-index)
# Tract_df.reset_index(inplace=True)

## Drop rows with missing shapefiles

In [26]:
df_geographic_unique = df_geographic_unique[(df_geographic_unique['geometry_BG'].notna())
                              #& (df_geographic_unique['geometry_county'].notna())
                              #& (df_geographic_unique['geometry_zipcode'].notna())
                              & (df_geographic_unique['geometry_tract'].notna())]

## Creating the intersections

In [27]:
import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [28]:
df_geographic_unique.columns

Index(['state', 'reportedZipCode', 'countyCode', 'censusTract',
       'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss',
       'yearOfLoss_2000_2021', 'geometry_lat_long', 'geometry_BG',
       'geometry_zipcode', 'geometry_state', 'geometry_county',
       'geometry_tract'],
      dtype='object')

In [29]:
len(df_geographic_unique)

11

In [30]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss_2000_2021', 'state', 'geometry_zipcode',
                                       'geometry_county', 'geometry_tract','geometry_BG','geometry_lat_long','state_geometry',
                                         'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 'stateInconsistent', 'latlongInconsistent', 'multiple', 'noOverlap', 'oneWrong'])

# Iterate through each row in BG_df_1990 and each row in lat_long_df to find intersections
for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_2000_2021']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    # Compute intersection geometry
    intersection_geometry = bg_geometry.intersection(lat_long_geometry).intersection(state_geometry).intersection(tract_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'state': [state],
            'geometry_zipcode': [zipcode_geometry],
            'geometry_county': [county_geometry],
            'geometry_tract': [tract_geometry],
            'geometry_BG': [bg_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'state_geometry': [state_geometry]
        })], ignore_index=True)

In [31]:
len(new_unit_df)

9

In [32]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = lat_long_geometry.intersection(state_geometry).intersection(tract_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit, 'cbgInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit, 'cbgInconsistent'] = 1

        

In [33]:
new_unit_df['cbgInconsistent'].sum()

0

In [34]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = bg_geometry.intersection(state_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'tractInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'tractInconsistent'] = 1

In [35]:
new_unit_df['tractInconsistent'].sum()

0

In [36]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = bg_geometry.intersection(tract_geometry).intersection(state_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 1

In [37]:
new_unit_df['countyInconsistent'].sum()

0

In [38]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = bg_geometry.intersection(tract_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 1

In [39]:
new_unit_df['stateInconsistent'].sum()

9

In [40]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = bg_geometry.intersection(tract_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 1

In [41]:
new_unit_df['latlongInconsistent'].sum()

1

In [ ]:
# for idx_unit, row_unit in new_unit_df.iterrows():
#     year = row_unit['year']
#     bg_id = row_unit['censusBlockGroupFips']
#     bg_geometry = row_unit['geometry_BG']
#     tract_id = row_unit['censusTract']
#     tract_geometry = row_unit['geometry_tract']
#     county_id = row_unit['countyCode']
#     county_geometry = row_unit['geometry_county']
#     state = row_unit['state']
#     state_geometry = row_unit['state_geometry']
#     lat_long_geometry = row_unit['geometry_lat_long']
#     lat = row_unit['latitude']
#     long = row_unit['longitude']
#     zipcode_geometry = row_unit['geometry_zipcode']
#     zipcode = row_unit['reportedZipCode']
    

#     intersection_geometry = bg_geometry.intersection(lat_long_geometry).intersection(tract_geometry).intersection(county_geometry).intersection(state_geometry)

#     if intersection_geometry is None or intersection_geometry.is_empty:
#         new_unit_df.at[idx_unit,'zipInconsistent'] = 0
#     else:
#         new_unit_df.at[idx_unit,'zipInconsistent'] = 1

In [ ]:
#print(new_unit_df['zipInconsistent'].sum())

In [42]:
mask = (new_unit_df['latlongInconsistent'] +   new_unit_df['stateInconsistent']  + new_unit_df['tractInconsistent'] + new_unit_df['cbgInconsistent']) > 1
new_unit_df.loc[mask, 'multiple'] = 1
new_unit_df.loc[~mask, 'multiple'] = 0

print(sum(new_unit_df['multiple']))

1


In [43]:
new_unit_df.head()

,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss_2000_2021,state,geometry_zipcode,geometry_county,geometry_tract,geometry_BG,geometry_lat_long,state_geometry,cbgInconsistent,tractInconsistent,countyInconsistent,stateInconsistent,latlongInconsistent,multiple,noOverlap,oneWrong,year
0,NaN,NaN,36061000900,360610009001,40.7,-74.0,NaN,LA,None,None,"POLYGON ((-74.01119 40.706278, -74.01042 40.70...","POLYGON ((-74.01119 40.706278, -74.01042 40.70...","POLYGON ((-74.05 40.650000000000006, -73.95 40...","MULTIPOLYGON (((-88.8677 29.861551, -88.865659...",0,0,0,1,0,0,NaN,NaN,2000.0
1,NaN,NaN,28011950200,280119502001,33.8,-90.9,NaN,AR,None,None,"POLYGON ((-90.869559 33.830797, -90.8686259999...","POLYGON ((-91.04020799999999 33.89026, -91.040...","POLYGON ((-90.95 33.75, -90.85000000000001 33....","POLYGON ((-94.617919 36.499414, -94.6176488354...",0,0,0,1,1,1,NaN,NaN,2010.0
2,NaN,NaN,11001007200,110010072001,38.9,-77.0,NaN,VA,None,None,"POLYGON ((-76.992668 38.870244, -76.993822 38....","POLYGON ((-76.992668 38.870244, -76.993822 38....","POLYGON ((-77.05 38.85, -76.95 38.85, -76.95 3...","MULTIPOLYGON (((-75.742406 37.80835, -75.74151...",0,0,0,1,0,0,NaN,NaN,2000.0
3,NaN,NaN,36061001200,360610012003,40.7,-74.0,NaN,WA,None,None,"POLYGON ((-73.980577 40.716454999999996, -73.9...","POLYGON ((-73.982289 40.716975999999995, -73.9...","POLYGON ((-74.05 40.650000000000006, -73.95 40...","MULTIPOLYGON (((-122.33164 48.020556, -122.328...",0,0,0,1,0,0,NaN,NaN,2000.0
4,NaN,NaN,36061027500,360610275001,40.9,-73.9,NaN,MO,None,None,"POLYGON ((-73.936655 40.857254999999995, -73.9...","POLYGON ((-73.93787 40.855678999999995, -73.93...","POLYGON ((-73.95 40.85, -73.85000000000001 40....","POLYGON ((-95.773549 40.578205, -95.768527 40....",0,0,0,1,0,0,NaN,NaN,2000.0


In [44]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'] ) == 0

new_unit_df['noOverlap'] = 0  # Default to 0
new_unit_df.loc[condition, 'noOverlap'] = 1

print(sum(new_unit_df['noOverlap']))

0


In [45]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'])  == 1

new_unit_df['oneWrong'] = 0  # Default to 0
new_unit_df.loc[condition, 'oneWrong'] = 1

print(sum(new_unit_df['oneWrong']))


8


In [46]:
# Initialize the count dictionary for 'Wrong' values
wrong_counts = {
    'lat_long_geometry': 0,
    'state_geometry': 0,
    'county_geometry': 0,
    'tract_geometry': 0,
    'bg_geometry': 0,
    'zip_geometry': 0  # added for zip codes
}

# Iterate through the DataFrame
for idx_unit, row_unit in new_unit_df.iterrows():
    if row_unit['oneWrong'] == 1:
        if row_unit['latlongInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'lat_long_geometry'] = 'Wrong'
            wrong_counts['lat_long_geometry'] += 1
        if row_unit['stateInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'state_geometry'] = 'Wrong'
            wrong_counts['state_geometry'] += 1
        if row_unit['countyInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'county_geometry'] = 'Wrong'
            wrong_counts['county_geometry'] += 1
        if row_unit['tractInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'tract_geometry'] = 'Wrong'
            wrong_counts['tract_geometry'] += 1
        if row_unit['cbgInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'bg_geometry'] = 'Wrong'
            wrong_counts['bg_geometry'] += 1


# Print the 'Wrong' counts
for category, count in wrong_counts.items():
    print(f"{category}: {count} 'Wrong' values")


lat_long_geometry: 0 'Wrong' values
state_geometry: 8 'Wrong' values
county_geometry: 0 'Wrong' values
tract_geometry: 0 'Wrong' values
bg_geometry: 0 'Wrong' values
zip_geometry: 0 'Wrong' values


In [47]:
new_unit_df['year'].value_counts()

2000.0    6
2010.0    3
Name: year, dtype: int64

In [48]:
df_geographic_unique['yearOfLoss_2000_2021'].value_counts()

2000    8
2010    3
Name: yearOfLoss_2000_2021, dtype: int64